In [1]:
# NS
#optiver-ver (for ensemble)

#optiver-ver-nn-modeling/inference-v1
#optiver-ver-rnn-modeling/inference-v1
#optiver-ver-lgb-modeling/inference-v1


import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from scipy.stats import hmean

from sklearn.ensemble import HistGradientBoostingRegressor
import itertools
import pickle
import joblib
from itertools import combinations
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Concatenate, GaussianNoise
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.experimental import CosineDecay
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.layers import concatenate,Dropout
import pickle
from tensorflow.keras.models import load_model
import os 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber
from tensorflow.keras.metrics import MeanAbsoluteError
from tensorflow.keras.callbacks import Callback
import random
from tensorflow.keras.layers import Input, Embedding, Lambda, Reshape, LSTM, Dense, BatchNormalization, Dropout, concatenate
from tensorflow.keras import backend as K
from tensorflow.keras.layers import ZeroPadding1D
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import RepeatVector

from tensorflow.keras.layers import Input, Embedding, Lambda, Reshape, LSTM, Dense, BatchNormalization, Dropout, concatenate
from tensorflow.keras import backend as K
from tensorflow.keras.layers import ZeroPadding1D, Activation
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import MaxPooling1D, AveragePooling1D
from tensorflow.keras.layers import Add

#---------------------------------------------------------------------- setup dashboard ------------------------------------------------------------

kaggle              = True
is_inference        = True
load_models         = True
run_pipeline        = False
train_models        = False
is_lgb              = True    #1
is_nn               = True    #2
is_rnn              = True   #3
simulation          = False
online_learning     = True

manage_memory       = True
memory_threshold    = 24576  #24GB

ens_models          = [0.5,0.25,0.25]

is_lgb_online              = True    #1
is_nn_online               = True    #2
is_rnn_online              = True   #3

#public-validation
# dates_train = [0,390]
# dates_test = [391,480]

#full-inference
dates_train = [0,480]
dates_test = [-1,-1]

num_models ={'lgb':1,'nn':1,'rnn':1} 


if kaggle:
    train_path = "/kaggle/input/optiver-trading-at-the-close/train.csv"
    models_path = "/kaggle/input/optiver-258-models/"
else:
    models_path = "optiver-inference-models/"
    train_path = "train.csv"

if dates_train[1]!=480:
    models_path = "vals/"
#---------------------------------------------------------------------- setup dashboard ------------------------------------------------------------


In [2]:
#@title params dashboard
lgb_params = {
            'learning_rate'     : 0.05,  #0.005,0.05
            'max_depth'         : 14, #14
            'n_estimators'      : 5000,
            'num_leaves'        : 31,    #511,31,1023
            'objective'         : 'mae',
            'subsample'         : .2, 
            'colsample_bytree'  : .3,
            'num_threads'       : 32,
            'device'            : 'gpu',
            # 'reg_alpha'         : 0.1,
            # 'reg_lambda'        : 4,
        }

nn_ep          = 60
nn_lr          = 0.001
nn_bs          = 2**15


rnn_ep         = 60
rnn_lr         = 0.001
rnn_bs         = 2**12
window_size    = 3

In [3]:
#@title read train

colab = False
if colab:
  from google.colab import drive
  drive.mount('/content/drive')
  train_path = '/content/drive/My Drive/optiver/train.csv'
  models_path = "/content/drive/My Drive/optiver/"
else:
    if not kaggle:
        from public_timeseries_testing_util import MockApi

pd.set_option('mode.chained_assignment', None)


def convert_price_cols_float32(df):

    # Columns containing 'price'
    price_columns = [col for col in df.columns if 'price' in col]
    df[price_columns] = df[price_columns].astype('float32')

    # Columns containing 'wap'
    wap_columns = [col for col in df.columns if 'wap' in col]
    df[wap_columns] = df[wap_columns].astype('float32')

    return df

train = pd.read_csv(train_path).drop(['row_id', 'time_id'], axis = 1)
nan_count = train['target'].isna().sum()
print(f"The 'target' column has {nan_count} NaN values.")

target_median = train['target'].median()
train['target'].fillna(target_median, inplace=True)

print(f"converting prices columns to float32 values.")
train = convert_price_cols_float32(train)
# ----------------------------- Reading train data -------------------------

The 'target' column has 88 NaN values.
converting prices columns to float32 values.


In [4]:
#@title functions

def split_by_date(df, dates):

    df_start, df_end = dates
    df = df[(df['date_id'] >= df_start) & (df['date_id'] <=df_end)].reset_index(drop=True)

    return df



def lag_function(df, columns_to_lag, numbers_of_days_to_lag):

    df_indexed = df.set_index(['stock_id', 'seconds_in_bucket', 'date_id'])
    
    for column_to_lag in columns_to_lag:
        for number_days_to_lag in numbers_of_days_to_lag:
            df_indexed[f'lag{number_days_to_lag}_{column_to_lag}'] = df_indexed.groupby(level=['stock_id', 'seconds_in_bucket'])[column_to_lag].shift(number_days_to_lag)
    
    df_indexed.reset_index(inplace=True)
    
    return df_indexed



def create_diff_lagged_features_within_date_revised(df, columns_to_lag, numbers_of_lag):
    df_copy = df.copy()

    # Store the new columns in a list
    new_columns = []

    # Iterate through each specified lag
    for lag in numbers_of_lag:
        # Create lagged dataframe once per lag value
        lagged_df = df.groupby(['stock_id', 'date_id'])[columns_to_lag].shift(periods=lag)
        
        # Iterate through each specified column
        for column in columns_to_lag:
            # Compute the new column
            new_col_name = f'{column}_diff_lag{lag}'
            new_column = df[column] - lagged_df[column]
            
            # Store the new column in the list
            new_columns.append(new_column.rename(new_col_name))

    # Concatenate the original dataframe with the new columns
    result_df = pd.concat([df_copy] + new_columns, axis=1)
    
    return result_df


def create_features_to_start_optimized(df, features_list):

    first_values_df = df.groupby(['stock_id', 'date_id'])[features_list].transform('first')

    for feature in features_list:
        feature_to_start_col_name = f'{feature}_to_start'
        df[feature_to_start_col_name] = df[feature] - first_values_df[feature]

    return df


def compute_imbalances(df_, columns, prefix = ''):
    """Computes the differences and imbalances for pairs of columns and stores them in the DataFrame."""
    df = df_.copy()
    for col1, col2 in combinations(columns, 2):
        
        # Sort the columns lexicographically to ensure consistent ordering
        col1, col2 = sorted([col1, col2])
        
        # Compute imbalance directly without creating a temporary difference column
        total = df[col1] + df[col2]
        imbalance_column_name = f'{col1}_{col2}_imb{prefix}'
        
        # Ensure we don't divide by zero
        df[imbalance_column_name] = (df[col1] - df[col2]).divide(total, fill_value=np.nan)

    return df

def compute_percentage_difference(df, columns, prefix = ''):

    df_copy = df.copy()
    
    # Iterate over all combinations of two different price columns
    for col1, col2 in combinations(columns, 2):
        # Sort the columns lexicographically to ensure consistent ordering
        col1, col2 = sorted([col1, col2])

        # Create a new column name based on the price columns
        new_col_name = f'pct_diff_{col1}_vs_{col2}_{prefix}'

        # Compute the percentage difference
        df_copy[new_col_name] = (df_copy[col1] - df_copy[col2]) / df_copy[col2] * 100

    return df_copy



def create_deviation_within_seconds(df, num_features):
    groupby_cols = ['date_id', 'seconds_in_bucket']
    new_columns = {}  # Dictionary to hold new columns

    for feature in num_features:
        grouped_median = df.groupby(groupby_cols)[feature].transform('median')
        deviation_col_name = f'deviation_from_median_{feature}'
        new_columns[deviation_col_name] = df[feature] - grouped_median

    # Concatenate all new columns at once
    df = pd.concat([df, pd.DataFrame(new_columns)], axis=1)
    return df


def create_cumsum_features(df, columns_to_compute):
    df_copy = df.copy()
    
    # Group by 'stock_id' and 'date_id' for cumulative sum calculation
    grouped = df_copy.groupby(['stock_id', 'date_id'])
    
    # Calculate cumulative sum for each column within each group
    for column in columns_to_compute:
        cumsum_col_name = f'{column}_cumsum'
        df_copy[cumsum_col_name] = grouped[column].cumsum()
    return df_copy


def save_pickle(data, file_path):
 
    # Create the directory if it doesn't exist
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Save the pickle file
    with open(file_path, 'wb') as file:
        pickle.dump(data, file)

    print(f"Data saved to {file_path}")
    #example: save_pickle(all_data, 'k8/all_data.pkl')

def load_pickle(file_path):

    # Load and return the data from the pickle file
    if os.path.exists(file_path):
        with open(file_path, 'rb') as file:
            data = pickle.load(file)
        return data
    else:
        raise FileNotFoundError(f"No such file: {file_path}")

    #example: loaded_data = load_pickle('k8/all_data.pkl')



In [5]:
#@title global
#---------------------------------- Global based on stock_id --------------------------------------

def aggregated_features_dic(df):


    global_feats = {}

    columns_to_aggregate= ['bid_size','ask_size']
    groupby_cols=['stock_id']

    def q25(x):
        return x.quantile(0.25)

    def q75(x):
        return x.quantile(0.75)

    # Define the aggregations
    aggregations = ['mean', 'median', 'std', 'min', 'max', q25, q75]

    # Prepare a dictionary to hold the aggregated Series


    # Perform aggregation for each column and operation
    for column in columns_to_aggregate:
        for agg in aggregations:
            # Define the aggregation function name
            if callable(agg):
                func_name = agg.__name__
            else:
                func_name = agg
            
            # Perform the aggregation
            agg_series = df.groupby(groupby_cols)[column].agg(agg)
            # Create a new feature name and add it to the dictionary
            new_feature_name = f"{func_name}_{column}"
            global_feats[new_feature_name] = agg_series

    global_feats["median_size"] = df.groupby("stock_id")["bid_size"].median() + df.groupby("stock_id")["ask_size"].median()
    global_feats["std_size"] = df.groupby("stock_id")["bid_size"].std() + df.groupby("stock_id")["ask_size"].std()
    global_feats["ptp_size"] = df.groupby("stock_id")["bid_size"].max() - df.groupby("stock_id")["bid_size"].min()
    global_feats["median_price"] = df.groupby("stock_id")["bid_price"].median() + df.groupby("stock_id")["ask_price"].median()
    global_feats["std_price"] = df.groupby("stock_id")["bid_price"].std() + df.groupby("stock_id")["ask_price"].std()
    global_feats["ptp_price"] = df.groupby("stock_id")["bid_price"].max() - df.groupby("stock_id")["ask_price"].min()


    return global_feats

aggregated_dic = aggregated_features_dic(train)

def map_global(df,dict):
    df_ = df.copy()
    for key, value in dict.items():
        df_[f"global_{key}"] = df_["stock_id"].map(value.to_dict())
    
    return df_
#---------------------------------- Global based on stock_id --------------------------------------



In [6]:
#@title helper functions


def flatten_outliers_y_train(y_train, lower_quantile=0.01, upper_quantile=0.99):

    lower_bound = np.quantile(y_train, lower_quantile)
    upper_bound = np.quantile(y_train, upper_quantile)

    # Cap values below the lower bound and above the upper bound
    y_train_flattened = np.clip(y_train, lower_bound, upper_bound)

    return y_train_flattened


def create_autocorrelation_features(df, columns, lags):

    df_copy = df.copy()
    
    for column in columns:
        for lag in lags:
            lagged_series = df_copy[column].shift(lag)
            df_copy[f'{column}_autocorr_lag{lag}'] = df_copy[column].corrwith(lagged_series)

    return df_copy


def calculate_stat_lag(df, num_lags):

    lags = [f'lag{i}_target' for i in range(1, num_lags + 1)]

    df['target_mean'] = df[lags].mean(axis=1)
    df['target_std_dev'] = df[lags].std(axis=1)
    df['target_variance'] = df[lags].var(axis=1)
    df['target_median'] = df[lags].median(axis=1)
    df['target_range'] = df[lags].max(axis=1) - df[lags].min(axis=1)

    return df


def calculate_stat(df, cols, prefix='prices'):

    df[f'{prefix}_mean'] = df[cols].mean(axis=1)
    df[f'{prefix}_std_dev'] = df[cols].std(axis=1)
    df[f'{prefix}_variance'] = df[cols].var(axis=1)
    df[f'{prefix}_median'] = df[cols].median(axis=1)
    df[f'{prefix}_range'] = df[cols].max(axis=1) - df[cols].min(axis=1)

    return df



In [7]:
#@title pipeline

raw_cols          = ['imbalance_size','matched_size','bid_size','ask_size','reference_price','far_price','near_price','bid_price','ask_price','wap','imbalance_buy_sell_flag'] 

columns_prices    = ['reference_price','far_price','near_price','bid_price','ask_price','wap']
columns_4prices   = ['reference_price','bid_price','ask_price','wap']

columns_sizes     = ['imbalance_size','matched_size','bid_size','ask_size']
columns_flag      = ['imbalance_buy_sell_flag'] 


diff_lags           = [1, 2, 3, 6, 12, 18, 24] 
diff_lags_extra     = [30, 36, 42, 48]

num_of_target_lags  = 12
target_lags         = list(range(1,num_of_target_lags+1))


def feature_engineering(df):
    
    df = df.copy()

    df['spread_eng']                            = df['ask_price'] - df['bid_price'] 
    df['volume_eng']                            = df['bid_size'] + df['ask_size']  
    df['volumne_imbalance_eng']                 = df['bid_size'] - df['ask_size']  
    
    df['imbalance_ratio']                       = df['imbalance_size'] / df['matched_size']  #(RM) Good

    df['price_spread_near_far']                 = df['near_price'] - df['far_price']   #RM (debatable)
    df['price_wap_difference_eng']              = df['reference_price'] - df['wap']    

    df['weighted_imbalance_eng']                = df['imbalance_size'] * df['imbalance_buy_sell_flag'] #very important


    df['bid_ask_ratio']                         = df['bid_size'] / df['ask_size'] #(RM) neutral
    df['imbalance_to_bid_ratio_eng']            = df['imbalance_size'] / df['bid_size']
    df['imbalance_to_ask_ratio_eng']            = df['imbalance_size'] / df['ask_size']
    df['matched_size_to_total_size_ratio_eng']  = df['matched_size'] / (df['bid_size'] + df['ask_size'])


    return df



def feature_pipeline(df):
    
    if df.empty:
        return pd.DataFrame()
        
    #--------------- connected ------------
    df = feature_engineering(df)
    df = compute_imbalances(df, columns_sizes,prefix='_sz_')
    df = compute_imbalances(df, columns_prices,prefix = '_pr_')



    eng_features       = [feature for feature in df.columns if "_eng" in feature]
    imb_features_all   = [feature for feature in df.columns if "_imb_" in feature]
    imb_features_price = [feature for feature in df.columns if "_pr_" in feature]
    imb_features_size  = [feature for feature in df.columns if "_sz_" in feature]

    #--------------- connected ------------


    diff_lag_cols = raw_cols + eng_features
    print(f"diff lagging {len(diff_lag_cols)} columns for {len(diff_lags)} lags.")
    df = create_diff_lagged_features_within_date_revised(df,diff_lag_cols,diff_lags)

    cumsum_columns = columns_sizes + imb_features_size + eng_features 
    print(f"cumsum for {len(cumsum_columns)} cols.")
    df = create_cumsum_features(df, cumsum_columns)

    deviation_cols = raw_cols + eng_features + imb_features_size #+ imb_features_price
    print(f"deviation {len(deviation_cols)} columns within seconds.")
    df = create_deviation_within_seconds(df,deviation_cols)

    print(f"lagging target column for {len(target_lags)} lags.")
    df = lag_function(df, ['target'], target_lags)

    df = map_global(df,aggregated_dic)

    df = calculate_stat_lag(df, num_lags=num_of_target_lags)

    df.replace([np.inf, -np.inf], np.nan, inplace=True)

    print("Done...")
    
    return df

In [8]:
#@title NN second pass
#------------------------------------------------- NN Second Pass Functions -----------------------------------------------

def make_predictions(models, X_test,model = 'nn'):
    if model == 'nn':
        all_predictions = [model.predict(X_test, batch_size=16384) for model in models]
    if model == 'lgb' or model == 'xgb' or model == 'cat':
        all_predictions = [model.predict(X_test) for model in models]
    prediction = np.mean(all_predictions, axis=0)
    return prediction

def set_all_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

class BestScoresCallback(Callback):
    def __init__(self):
        super().__init__()
        self.best_train_loss = float('inf')
        self.best_val_loss = float('inf')

    def on_epoch_end(self, epoch, logs=None):
        train_loss = logs.get('loss', float('inf'))
        val_loss = logs.get('val_loss', float('inf'))

        if train_loss < self.best_train_loss:
            self.best_train_loss = train_loss
        if val_loss < self.best_val_loss:
            self.best_val_loss = val_loss

    def on_train_end(self, logs=None):
        print(f"Best training loss: {self.best_train_loss}, Best validation loss: {self.best_val_loss}")


def second_pass_for_nn(df, numerical_features, categorical_features, is_inference=False):
    # Check if the DataFrame is empty
    global scaler,medians
    
    if df.empty:
        return None, None

    # Work on a copy of the DataFrame to avoid changing the original df
    df_copy = df.copy()


    # Standard scaling for numerical features
    if is_inference:
        df_copy.fillna(medians, inplace=True)
        df_copy[numerical_features] = scaler.transform(df_copy[numerical_features])

    # Preprocess Data
    df_copy['seconds_in_bucket'] = df_copy['seconds_in_bucket'] / 10
    df_copy['imbalance_buy_sell_flag'] += 1 

    # Create input vector
    X_cat = [df_copy[cat].values.reshape(-1, 1) for cat in categorical_features]
    X_num = df_copy[numerical_features].values
    

    y = df_copy['target'].values
        
    return [X_num] + X_cat, y

#------------------------------------------------- NN Second Pass Functions -----------------------------------------------

In [9]:
#@title NN model 

def create_model(categorical_features,numerical_features,initial_learning_rate=0.001):

    inputs = []
    embeddings = []

    categorical_uniques = {}
    categorical_uniques['seconds_in_bucket'] = 55

    embedding_dim = {}
    embedding_dim['seconds_in_bucket'] = 10


    input_num = Input(shape=(len(numerical_features),), name="numerical_input")
    inputs.append(input_num)

    for cat_feature in categorical_features:
        vocab_size = categorical_uniques[cat_feature]
        input_cat = Input(shape=(1,), name=f"input_{cat_feature}")
        embedding = Embedding(vocab_size, embedding_dim[cat_feature], input_length=1, name=f"embedding_{cat_feature}")(input_cat)
        flattened_embedding = Flatten(name=f"flatten_{cat_feature}")(embedding)
        inputs.append(input_cat)
        embeddings.append(flattened_embedding)

    dense_output = concatenate(embeddings + [input_num])

    dense_sizes = [512, 256, 128, 64, 32]

    for size in dense_sizes:
        dense_output = Dense(size, activation='swish')(dense_output)
        dense_output = BatchNormalization()(dense_output)
        dense_output = Dropout(0.4)(dense_output)

    output = Dense(1)(dense_output)


    lr_schedule = ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=3000,           
    decay_rate=0.5,
    staircase=True)

    model = Model(inputs, output)
    optimizer = Adam(learning_rate=lr_schedule)

    model.compile(optimizer=optimizer, loss = "mean_absolute_error")

    return model

In [10]:
#@title RNN second pass

def precompute_sequences(stock_data, window_size, rnn_numerical_features, rnn_categorical_features):
    # Convert DataFrame columns to NumPy arrays
    stock_data_num = stock_data[rnn_numerical_features].values
    stock_data_cat = stock_data[rnn_categorical_features].values

    # Pre-compute all sequences
    all_sequences_num = [stock_data_num[max(0, i - window_size + 1):i + 1] for i in range(len(stock_data))]
    all_sequences_cat = [stock_data_cat[max(0, i - window_size + 1):i + 1] for i in range(len(stock_data))]

    # Add padding if necessary
    padded_sequences_num = [np.pad(seq, ((window_size - len(seq), 0), (0, 0)), 'constant') for seq in all_sequences_num]
    padded_sequences_cat = [np.pad(seq, ((window_size - len(seq), 0), (0, 0)), 'constant') for seq in all_sequences_cat]

    # Combine numerical and categorical features
    combined_sequences = np.array([np.concatenate([num, cat], axis=-1) 
                                   for num, cat in zip(padded_sequences_num, padded_sequences_cat)])

    # Extract targets
    targets = stock_data['target'].values

    return combined_sequences, targets

def get_sequence(precomputed_data, time_step):
    combined_sequences, targets = precomputed_data
    return combined_sequences[time_step], targets[time_step]



def create_batches(data, window_size, rnn_numerical_features, rnn_categorical_features, max_time_steps=55):
    
    grouped = data.groupby(['stock_id', 'date_id'])
    all_batches = []
    all_targets = []

    for _, group in tqdm(grouped, desc="Processing groups"):

        # Precompute sequences for the current group
        precomputed_data = precompute_sequences(group, window_size, rnn_numerical_features, rnn_categorical_features)

        # Initialize containers for group sequences and targets
        group_sequences = []
        group_targets = []

        # Iterate over the time steps and retrieve precomputed sequences
        for time_step in range(max_time_steps):
            sequence, target = get_sequence(precomputed_data, time_step)
            if sequence.size > 0: 
                group_sequences.append(sequence)
                group_targets.append(target)

        # Extend the main batches with the group's sequences and targets
        all_batches.extend(group_sequences)
        all_targets.extend(group_targets)

    return all_batches, all_targets




def compute_last_sequence(group, window_size, rnn_numerical_features, rnn_categorical_features):
    # Convert DataFrame columns to NumPy arrays
    stock_data_num      = group[rnn_numerical_features].values
    stock_data_cat      = group[rnn_categorical_features].values
    stock_data_target   = group['target'].values

    # Find the index of the target second
    target_index = len(group) - 1

    # Extract the sequence for the target index
    sequence_num = stock_data_num[max(0, target_index - window_size + 1):target_index + 1]
    sequence_cat = stock_data_cat[max(0, target_index - window_size + 1):target_index + 1]

    # Add padding if necessary
    padded_sequence_num = np.pad(sequence_num, ((window_size - len(sequence_num), 0), (0, 0)), 'constant')
    padded_sequence_cat = np.pad(sequence_cat, ((window_size - len(sequence_cat), 0), (0, 0)), 'constant')

    # Combine numerical and categorical features
    combined_sequence = np.concatenate([padded_sequence_num, padded_sequence_cat], axis=-1)

    # Extract target
    target = stock_data_target[-1]

    return combined_sequence, target


def create_last_batches(data, window_size, rnn_numerical_features, rnn_categorical_features):
    
    grouped = data.groupby(['stock_id'])
    all_batches = []
    all_targets = []

    for _, group in grouped:
        # Compute the sequence for the last data point in the current group
        last_sequence, last_target = compute_last_sequence(group, window_size, rnn_numerical_features, rnn_categorical_features)

        # Check if the sequence is valid (i.e., not empty)
        if last_sequence.size > 0: 
            all_batches.append(last_sequence)
            all_targets.append(last_target)

    return all_batches, all_targets





def second_pass_for_rnn(df, rnn_numerical_features, rnn_categorical_features, window_size, is_inference=False):
    # Check if the DataFrame is empty
    global rnn_scaler,rnn_medians
    
    if df.empty:
        return None, None

    # Work on a copy of the DataFrame to avoid changing the original df
    df_copy = df.copy()

    # Standard scaling for numerical features
    if is_inference:
        df_copy.fillna(rnn_medians, inplace=True)
        df_copy[rnn_numerical_features] = rnn_scaler.transform(df_copy[rnn_numerical_features])
        
    # Preprocess Data
    df_copy['seconds_in_bucket'] = df_copy['seconds_in_bucket'] / 10
    df_copy['imbalance_buy_sell_flag'] += 1 

    if is_inference:
        df_copy_batches, df_copy_targets = create_last_batches(df_copy, window_size, rnn_numerical_features, rnn_categorical_features)
    else:
        df_copy_batches, df_copy_targets = create_batches(df_copy, window_size, rnn_numerical_features, rnn_categorical_features)

    df_copy_batches = np.array(df_copy_batches)
    df_copy_targets = np.array(df_copy_targets)


    return df_copy_batches, df_copy_targets


def online_pass_for_rnn(df, rnn_numerical_features, rnn_categorical_features, window_size):
    # Check if the DataFrame is empty
    global rnn_scaler,rnn_medians
    
    if df.empty:
        return None, None

    # Work on a copy of the DataFrame to avoid changing the original df
    df_copy = df.copy()

    # Standard scaling for numerical features
    df_copy.fillna(rnn_medians, inplace=True)
    df_copy[rnn_numerical_features] = rnn_scaler.transform(df_copy[rnn_numerical_features])
        
    # Preprocess Data
    df_copy['seconds_in_bucket'] = df_copy['seconds_in_bucket'] / 10
    df_copy['imbalance_buy_sell_flag'] += 1 


    grouped = df_copy.groupby(['stock_id'])
    all_batches = []
    all_targets = []

    for _, group in tqdm(grouped, desc="Processing groups"):
        # Precompute sequences for the current group
        precomputed_data = precompute_sequences(group, window_size, rnn_numerical_features, rnn_categorical_features)

        # Initialize containers for group sequences and targets
        group_sequences = []
        group_targets = []

        # Iterate over the time steps and retrieve precomputed sequences
        for time_step in range(55):
            sequence, target = get_sequence(precomputed_data, time_step)
            if sequence.size > 0: 
                group_sequences.append(sequence)
                group_targets.append(target)

        # Extend the main batches with the group's sequences and targets
        all_batches.extend(group_sequences)
        all_targets.extend(group_targets)

    df_batches = np.array(all_batches)
    df_targets = np.array(all_targets)


    return df_batches, df_targets



In [11]:
#@title RNN model

def apply_conv_layers(input_layer, kernel_sizes, filters=16, do_ratio=0.5):
    conv_outputs = []

    for kernel_size in kernel_sizes:
        conv_layer = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', padding='same')(input_layer)
        conv_layer = BatchNormalization()(conv_layer)
        conv_layer = Dropout(do_ratio)(conv_layer)

        shortcut = conv_layer

        conv_layer = Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(conv_layer)
        conv_layer = BatchNormalization()(conv_layer)
        conv_layer = Activation('relu')(conv_layer)

        # Add the output of the first Conv1D layer
        conv_layer = Add()([conv_layer, shortcut])
        conv_outputs.append(conv_layer)


    concatenated_conv = Concatenate(axis=-1)(conv_outputs)
    flattened_conv_output = Flatten()(concatenated_conv)

    return flattened_conv_output

def create_rnn_model_with_residual(window_size, numerical_features, initial_learning_rate=0.001):

    categorical_features = 'seconds_in_bucket'
    categorical_uniques  = { 'seconds_in_bucket' : 55}
    embedding_dim        = {'seconds_in_bucket' : 10}

    input_layer = Input(shape=(window_size, len(numerical_features) + 1), name="combined_input")

    # Split the input into numerical and categorical parts
    numerical_input = Lambda(lambda x: x[:, :, :-1], name="numerical_part")(input_layer)
    categorical_input = Lambda(lambda x: x[:, :, -1:], name="categorical_part")(input_layer)

    first_numerical = Lambda(lambda x: x[:, 0])(numerical_input)


    # diffrentiate layers
    def create_difference_layer(lag):
        return Lambda(lambda x: x[:, lag:, :] - x[:, :-lag, :], name=f"difference_layer_lag{lag}")

    difference_layers = []
    for lag in range(1, window_size):
        diff_layer = create_difference_layer(lag)(numerical_input)
        padding = ZeroPadding1D(padding=(lag, 0))(diff_layer)  # Add padding to the beginning of the sequence
        difference_layers.append(padding)
    combined_diff_layer = Concatenate(name="combined_difference_layer")(difference_layers)


    # Embedding for categorical part
    vocab_size, embedding_dim = categorical_uniques[categorical_features], embedding_dim[categorical_features]
    embedding = Embedding(vocab_size, embedding_dim, input_length=window_size)(categorical_input)
    embedding = Reshape((window_size, -1))(embedding)

    first_embedding = Lambda(lambda x: x[:, 0])(embedding)

    # Concatenate numerical input and embedding
    # conv_input = concatenate([enhanced_numerical_input, embedding], axis=-1)

    kernel_sizes = [2,3]
    do_ratio = 0.4

    flattened_conv_output = apply_conv_layers(numerical_input, kernel_sizes, do_ratio=do_ratio)
    flattened_conv_output_cat = apply_conv_layers(embedding, kernel_sizes, do_ratio=do_ratio)
    flattened_conv_output_diff = apply_conv_layers(combined_diff_layer, kernel_sizes, do_ratio=do_ratio)


    dense_output = Concatenate(axis=-1)([flattened_conv_output,flattened_conv_output_cat,flattened_conv_output_diff, Reshape((-1,))(combined_diff_layer),first_numerical,first_embedding])

    dense_sizes = [512, 256, 128, 64, 32]
    do_ratio = 0.5
    for size in dense_sizes:
        dense_output = Dense(size, activation='swish')(dense_output)
        dense_output = BatchNormalization()(dense_output)
        dense_output = Dropout(do_ratio)(dense_output)

    # Output layer
    output = Dense(1, name='output_layer')(dense_output)

    # Learning rate schedule
    lr_schedule = ExponentialDecay(
        initial_learning_rate=initial_learning_rate,
        decay_steps=10000,
        decay_rate=0.7,
        staircase=True)

    # Create and compile the model
    model = Model(inputs=input_layer, outputs=output)
    optimizer = Adam(learning_rate=lr_schedule)

    model.compile(optimizer=optimizer, loss="mean_absolute_error")

    return model

In [12]:
#@title runnig pipeline

excluded_columns = ['row_id', 'date_id', 'time_id', 'target','stock_return']  

if run_pipeline:

    train_eng = feature_pipeline(train)
    
    
    if is_nn:
        excluded_columns = excluded_columns + ['stock_id']

        features = [col for col in train_eng.columns if col not in excluded_columns]
        categorical_features =  ['seconds_in_bucket']
        numerical_features = [feat for feat in features if feat not in categorical_features]
        print("we have {} numerical and {} categorical".format(len(numerical_features),len(categorical_features)))



        scaler = StandardScaler()
        medians = train_eng.median()
        
        train_eng.fillna(medians, inplace=True)
        train_eng[numerical_features] = scaler.fit_transform(train_eng[numerical_features])

    
        all_data = {
            "scaler": scaler,
            "medians": medians,
            "categorical_features": categorical_features,
            "numerical_features": numerical_features
        }

        save_pickle(all_data, f'{models_path}all_data.pkl')
        print("Pipline Done!")

    if is_rnn:
        excluded_columns = excluded_columns + ['stock_id']

        features = [col for col in train_eng.columns if col not in excluded_columns]
        rnn_categorical_features =  ['seconds_in_bucket']
        rnn_numerical_features = [feat for feat in features if feat not in rnn_categorical_features]
        print("we have {} numerical and {} categorical".format(len(rnn_numerical_features),len(rnn_categorical_features)))


        rnn_scaler = StandardScaler()
        rnn_medians = train_eng.median()
        
        train_eng.fillna(rnn_medians, inplace=True)
        train_eng[rnn_numerical_features] = rnn_scaler.fit_transform(train_eng[rnn_numerical_features])

    
        rnn_all_data = {
            "rnn_scaler": rnn_scaler,
            "rnn_medians": rnn_medians,
            "rnn_categorical_features": rnn_categorical_features,
            "rnn_numerical_features": rnn_numerical_features
        }

        save_pickle(rnn_all_data, f'{models_path}rnn_all_data.pkl')
        print("Pipline Done!")

    if is_lgb:
        lgb_features = [col for col in train_eng.columns if col not in excluded_columns]
        categorical_features = ['seconds_in_bucket']

        print("we have {} lgb features".format(len(lgb_features)))
        
    train_data       = split_by_date(train_eng, dates_train)
    test_data        = split_by_date(train_eng, dates_test)
    print("number of dates in train = {} , number of dates in test {}".format (train_data['date_id'].nunique(),test_data['date_id'].nunique()))

    cleaning = False
    if cleaning:
        import gc
        #del train
        del train_eng
        gc.collect()
    



In [13]:
#@title TPU
try:
    # Create a TPUClusterResolver
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    # Connect to the TPU cluster
    tf.config.experimental_connect_to_cluster(tpu)
    # Initialize the TPU system
    tf.tpu.experimental.initialize_tpu_system(tpu)
    # Create a TPUStrategy for distributed training
    tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    tpu_strategy = None  # No TPU found


In [14]:
#@title prepare train models
if train_models:

    if test_data.empty:
        print("--------------test data is empty so adjusting the last date for test-----------------")
        test_data = train_data.query("date_id == 480").copy()

    if is_lgb:


        X_train, y_train = train_data[lgb_features], train_data['target']
        X_test, y_test = test_data[lgb_features],test_data['target']
        train_set = lgb.Dataset(X_train, label=y_train,categorical_feature=categorical_features,free_raw_data=False)
        test_set = lgb.Dataset(X_test, label=y_test,categorical_feature=categorical_features,free_raw_data=False)

    if is_nn:

        X_train, y_train    = second_pass_for_nn(train_data,numerical_features,categorical_features)
        X_test, y_test      = second_pass_for_nn(test_data,numerical_features,categorical_features)
    
    if is_rnn:

        train_batches, train_targets = second_pass_for_rnn(train_data, rnn_numerical_features, rnn_categorical_features, window_size)
        test_batches, test_targets  = second_pass_for_rnn(test_data, rnn_numerical_features, rnn_categorical_features, window_size)
        print(f"train batches shape:{train_batches.shape}")


In [15]:
#@title training models
# train_data = flatten_outliers(train_data, 'target', lower_quantile=0.01, upper_quantile=0.99)
if train_models:
        
        directory = os.path.dirname(models_path)
        if not os.path.exists(directory):
            os.makedirs(directory)

        if is_lgb:
            lgb_models = []
            for i in range(num_models['lgb']):
                rnd_state=42+i
                print(f"Training model {i+1} out of {num_models['lgb']} with seed {rnd_state}")
                print("---------------------------------------")

                lgb_params['random_state'] = rnd_state
                lgb_model = lgb.train(lgb_params, train_set,init_model=None, valid_sets=[train_set, test_set],verbose_eval=50, early_stopping_rounds=250)
                lgb_model.save_model(f'{models_path}model_lgb_{i}.txt')
                lgb_models.append(lgb_model)
                        
                if dates_train[1]!=480:
                    pred = lgb_model.predict(X_test)
                    mae = mean_absolute_error(test_data['target'] , pred)
                    print(f"Mean Absolute Error on test data: {mae:.5f}")

            if dates_train[1]!=480:
                predictions =  make_predictions(lgb_models, test_data[lgb_features],model = 'lgb')                   
                print(f"LGB Ensemble Mean Absolute Error: {mean_absolute_error(test_data['target'], predictions):.5f}")
   
        if is_nn:

            callbacks = [BestScoresCallback()]  # Always include BestScoresCallback
            if dates_train[1] != 480:
                early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
                callbacks.append(early_stopping)

            if tpu_strategy:
                with tpu_strategy.scope():

                    nn_models = []
                    for i in range(num_models['nn']):
                        print(f"Training nn model {i+1} out of {num_models['nn']} with seed {42+i}")
                        print("---------------------------------------")
                        set_all_seeds(42+i)
                        
                        nn_model = create_model(categorical_features, numerical_features, initial_learning_rate=nn_lr)
                        history = nn_model.fit(X_train, y_train, validation_data=(X_test, y_test),epochs=nn_ep, batch_size=nn_bs, callbacks=callbacks)

                        print("---------------------------------------")
                        nn_model.save(f'{models_path}swish_model_seed_{i}.h5')
                        nn_models.append(nn_model)
                        
                    predictions =  make_predictions(nn_models, X_test,model ='nn')                        

                    print(f"Ensemble Mean Absolute Error: {mean_absolute_error(y_test, predictions):.4f}")

        if is_rnn:

            callbacks = [BestScoresCallback()]  # Always include BestScoresCallback
            if dates_train[1] != 480:
                early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
                callbacks.append(early_stopping)

            if True or tpu_strategy:
                # with tpu_strategy.scope():

                    rnn_models = []
                    for i in range(num_models['rnn']):

                        print(f"Training rnn model {i+1} out of {num_models['rnn']} with seed {42+i}")
                        print("---------------------------------------")
                        set_all_seeds(42+i)

                        rnn_model = create_rnn_model_with_residual(window_size, rnn_numerical_features, initial_learning_rate=rnn_lr)
                        history = rnn_model.fit(train_batches, train_targets, validation_data=(test_batches, test_targets), epochs=rnn_ep, batch_size=rnn_bs, callbacks=callbacks)
                        print("---------------------------------------")
                        rnn_model.save(f'{models_path}rnn_model_seed_{i}.h5')
                        rnn_models.append(rnn_model)

                    predictions =  make_predictions(rnn_models, test_batches,model = 'nn')                        
                    print(f"Ensemble Mean Absolute Error: {mean_absolute_error(test_targets, predictions):.4f}")


In [16]:
#@title comments

#=================================================  258 feat  ==============================================

#391-480 public-validation seed 42,43: lr:0.05,31 <sub,col 0.2,0.3>  depth = 14
#----------- [3281]	training's l1: 6.10386	valid_1's l1: 5.86598
#----------- [2840]	training's l1: 6.12476	valid_1's l1: 5.86619
#LGB Ensemble Mean Absolute Error: 5.8637


#391-480 public-validation seed 42,43: lr:0.005,511 <sub,col 0.2,0.3>  depth = 14
# ----------------- [5670]	training's l1: 5.91513	valid_1's l1: 5.86133
#------------------ [4606]	training's l1: 5.95538	valid_1's l1: 5.8615
# LGB Ensemble Mean Absolute Error: 5.8610


#391-480 public-validation seed 42,43: lr:0.005,1023 <sub,col 0.2,0.3>  depth = 14
# ----------------- [4028]	training's l1: 5.79751	valid_1's l1: 5.86096





#=================================================  253 feat  ==============================================

#391-480 public-validation seed 42,43: lr:0.05,31 <sub,col 0.2,0.3>  depth = 14
#----------- [3058]	training's l1: 6.11684	valid_1's l1: 5.8669
#----------- [3453]	training's l1: 6.09743	valid_1's l1: 5.86741
#LGB Ensemble Mean Absolute Error: 5.8647


#391-480 public-validation seed 42,43: lr:0.005,511 <sub,col 0.2,0.3>  depth = 14
# ----------------- [5242]	training's l1: 5.92851	valid_1's l1: 5.86238
#------------------ [6000]	training's l1: 5.90175	valid_1's l1: 5.86203

#LGB Ensemble Mean Absolute Error: 5.8618


In [17]:
#@title importance
imp = False
if imp:
    importances = lgb_model.feature_importance()
    feature_names = lgb_model.feature_name()

    # Creating a DataFrame for easy manipulation
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importances
    })
    importance_df.to_csv("importance.csv")

In [18]:
#@title timeseries functions

def clean_format(df):
    df['target'] = df['target'].astype('float64')
    for feat in ['stock_id','date_id','seconds_in_bucket']:
        df[feat] = df[feat].astype('int64')
    return df

def clean_up(df):
    df = df[['stock_id','revealed_date_id','seconds_in_bucket','revealed_target']].rename(columns={'revealed_date_id': 'date_id', 'revealed_target': 'target'})
    df['target'].fillna(-0.06020069, inplace=True)
    df = clean_format(df)
    return df

def manage_buffer(buffer, df, max_lag):

    # df['iteration'] = df['iteration'].max()  # Ensure the iteration number is consistent in the new df chunk
    if buffer.empty:
        return df.copy()
    elif buffer['iteration'].nunique() < max_lag:
        return pd.concat([buffer, df], ignore_index=True)
    else:
        oldest_iteration = buffer['iteration'].min()
        buffer = buffer[buffer['iteration'] > oldest_iteration]
        return pd.concat([buffer, df], ignore_index=True)
    

def append_target_lags(df, target_buffer, lags):

    column_to_lag = 'target'
    df_lagged = df.copy()

    for lag in lags:

        temp_df = target_buffer.copy()
        temp_df['date_id'] = temp_df['date_id'] + lag
        
        temp_df.rename(columns={column_to_lag: f'lag{lag}_{column_to_lag}'}, inplace=True)
        
        df_lagged = pd.merge(df_lagged, temp_df[['stock_id', 'date_id', 'seconds_in_bucket', f'lag{lag}_{column_to_lag}']], 
                             on=['stock_id', 'date_id', 'seconds_in_bucket'], 
                             how='left')
        
    return df_lagged


def timeseries_lag_features(data, columns_to_lag, numbers_of_lag):
    for col in columns_to_lag:
        for lag in numbers_of_lag:
            data[f'{col}_lag{lag}'] = data.groupby(['stock_id'])[col].shift(lag)
    return data


def timeseries_diff_lag_features(data, columns_to_lag, numbers_of_lag):
    # Store new columns in a dictionary before joining them to the original DataFrame
    new_columns = {}

    # Iterate over each group defined by 'stock_id' to reduce groupby operations
    grouped_data = data.groupby('stock_id')

    for col in columns_to_lag:
        for lag in numbers_of_lag:
            diff_col_name = f'{col}_diff_lag{lag}'

            # Compute the lagged difference for each group
            new_columns[diff_col_name] = data[col] - grouped_data[col].shift(lag)

    # Concatenate all new columns and join with the original DataFrame
    new_columns_df = pd.DataFrame(new_columns)
    result_df = pd.concat([data, new_columns_df], axis=1)

    return result_df


def timeseries_cumsum_features(df, columns_to_compute):
    df_copy = df.copy()
    
    # Group by 'stock_id' and 'date_id' for cumulative sum calculation
    grouped = df_copy.groupby(['stock_id'])
    
    # Calculate cumulative sum for each column within each group
    for column in columns_to_compute:
        cumsum_col_name = f'{column}_cumsum'
        df_copy[cumsum_col_name] = grouped[column].cumsum()
        
    return df_copy


def timeseries_deviation_within_seconds(df, num_features):
    # Calculating the median for each numerical feature
    medians = df[num_features].median()

    # Calculate deviations in a vectorized manner
    deviation_cols = {f'deviation_from_median_{feature}': df[feature] - medians[feature] for feature in num_features}
    df = df.assign(**deviation_cols)

    return df


import psutil

def memory_limit_exceeded(threshold_in_mb):

    process = psutil.Process()
    current_memory_usage = process.memory_info().rss / (2**20)  # Convert to MB
    return current_memory_usage > threshold_in_mb

In [19]:
if kaggle:
    import gc
    # remove train_small and train and clear the memory
    del train
    gc.collect()

In [20]:
#@title is inference
if is_inference:

    if online_learning:
        
        #lgb
        lgb_params['learning_rate'] = 0.001 #0.001
        lgb_params['n_estimators']  = 20    #10,20
        lgb_params['num_leaves']    = 127    #127
        lgb_params['device']        = 'cpu'

        #nn
        online_nn_lr_rate    = 2e-4 #2e-4
        online_nn_batch_size = 16384 #16384
        online_nn_epochs     = 4     #2,4

        #rnn
        online_rnn_lr_rate    = 2e-4 #2e-4
        online_rnn_batch_size = 2**12 #2**12 
        online_rnn_epochs     = 4      #2



    if kaggle:
        import optiver2023
        env = optiver2023.make_env()
        iter_test = env.iter_test()
    else:
        env = MockApi()
        iter_test = env.iter_test()
    
    if load_models:
        


        if is_lgb:
            print("loading lgb models...")
            lgb_models = []
            for i in range(num_models['lgb']):
                loaded_model = lgb.Booster(model_file=f'{models_path}model_lgb_{i}.txt')
                lgb_models.append(loaded_model)
            
            lgb_features = lgb_models[0].feature_name()
            print("Done!")


        if is_nn:

            print("loading nn models...")
            loaded_data          = load_pickle(f'{models_path}all_data.pkl')
            scaler               = loaded_data["scaler"]
            medians              = loaded_data["medians"]
            categorical_features = loaded_data["categorical_features"]
            numerical_features   = loaded_data["numerical_features"]

            nn_models = []
            for i in range(num_models['nn']):
                loaded_model = load_model(f"{models_path}swish_model_seed_{i}.h5")
                nn_models.append(loaded_model)
            print("Done!")

        if is_rnn:

            print("loading rnn models...")
            loaded_data          = load_pickle(f'{models_path}rnn_all_data.pkl')
            rnn_scaler               = loaded_data["rnn_scaler"]
            rnn_medians              = loaded_data["rnn_medians"]
            rnn_categorical_features = loaded_data["rnn_categorical_features"]
            rnn_numerical_features   = loaded_data["rnn_numerical_features"]

            rnn_models = []
            for i in range(num_models['rnn']):
                loaded_model = load_model(f"{models_path}rnn_model_seed_{i}.h5")
                rnn_models.append(loaded_model)
            print("Done!")



    #---------------------------buffer management ---------------------------
    max_target_lag  = num_of_target_lags
    i               = 0
    target_counter  = 0
    target_buffer   = pd.DataFrame()
    total_test      = pd.DataFrame()
    daily_online    = pd.DataFrame()
    #---------------------------buffer management ---------------------------


    for (test, revealed_targets, sample_prediction) in iter_test:
        
        try:
            test = convert_price_cols_float32(test)


            if test.seconds_in_bucket.iloc[0] == 0:
            
                #----------------------------- storing previous targets -------------------------

                try:
                    revealed_targets = clean_up(revealed_targets)
                    revealed_targets['iteration'] = target_counter
                    target_buffer = manage_buffer(target_buffer, revealed_targets, max_target_lag)
                    target_counter += 1
                    
                except Exception as e:
                    print(f"An error occurred: {e}")                


                if manage_memory:
                    if memory_limit_exceeded(24576):
                        is_rnn_online = False
                    if memory_limit_exceeded(26276):
                        is_nn_online  = False
                    if memory_limit_exceeded(28276):
                        is_lgb_online = False


                try:
                    if not daily_online.empty and online_learning:

                        daily_online.drop(columns='target', inplace=True)
                        daily_online = pd.merge(daily_online, revealed_targets[['stock_id', 'date_id', 'seconds_in_bucket', 'target']], 
                                                    on=['stock_id', 'date_id', 'seconds_in_bucket'], 
                                                    how='inner')


                        if is_lgb and is_lgb_online:
                            online_lgbs = []

                            for lgb_model in lgb_models:
                                new_data = lgb.Dataset(daily_online[lgb_features], label=daily_online['target'])
                                online_lgb = lgb.train(lgb_params, new_data, init_model=lgb_model,valid_sets=[new_data,new_data],keep_training_booster=True,verbose_eval=1)
                                online_lgbs.append(online_lgb)

                            #update the lgb_models
                            lgb_models = online_lgbs.copy()

                        if is_nn and is_nn_online:

                            X_online, y_online          = second_pass_for_nn(daily_online,numerical_features,categorical_features,is_inference=True)

                            for nn_model in nn_models:
                                lr_schedule = ExponentialDecay(initial_learning_rate=online_nn_lr_rate,decay_steps=10000,decay_rate=0.96,staircase=True)
                                optimizer = Adam(learning_rate=lr_schedule)
                                nn_model.compile(optimizer=optimizer, loss = "mean_absolute_error")
                                nn_model.fit(X_online, y_online, epochs= online_nn_epochs, batch_size=online_nn_batch_size) 
                                K.clear_session()
                                gc.collect()  

                        if is_rnn and is_rnn_online:

                            online_batches, online_targets = online_pass_for_rnn(daily_online, rnn_numerical_features, rnn_categorical_features, window_size)
                            
                            for rnn_model in rnn_models:
                                lr_schedule = ExponentialDecay( initial_learning_rate=online_rnn_lr_rate, decay_steps=10000, decay_rate=0.8,staircase=True)
                                optimizer = Adam(learning_rate=lr_schedule)
                                rnn_model.compile(optimizer=optimizer, loss = "mean_absolute_error")
                                rnn_model.fit(online_batches, online_targets, epochs=online_rnn_epochs, batch_size=online_rnn_batch_size)
                                K.clear_session()
                                gc.collect()  

                except Exception as e:
                    print(f"An error occurred: {e}")

                daily_online = pd.DataFrame()
                #----------------------------- storing previous targets -------------------------


            test['target'] = 0

            #--------------- connected ------------
            test = feature_engineering(test)
            test = compute_imbalances(test, columns_sizes,prefix='_sz_')
            test = compute_imbalances(test, columns_prices,prefix = '_pr_')

            test = append_target_lags(test, target_buffer, target_lags)
            
            test = calculate_stat_lag(test, num_lags=num_of_target_lags)

            eng_features       = [feature for feature in test.columns if "_eng" in feature]
            imb_features_all   = [feature for feature in test.columns if "_imb_" in feature]
            imb_features_price = [feature for feature in test.columns if "_pr_" in feature]
            imb_features_size  = [feature for feature in test.columns if "_sz_" in feature]
            #--------------- connected ------------

            #------------------------------------  this for deviation within seconds  -------------------------
            deviation_cols = raw_cols + eng_features + imb_features_size + imb_features_price
            test = timeseries_deviation_within_seconds(test,deviation_cols)

            test = map_global(test,aggregated_dic)

            #------------------------ get the full data up to current seconds in bucket in date-----------------------------------
            if not daily_online.empty:  
                full_data = pd.concat([daily_online[test.columns], test], ignore_index=True)
            else:
                full_data = test


            diff_lag_cols = raw_cols + eng_features 
            full_data = timeseries_diff_lag_features(full_data, diff_lag_cols, diff_lags)

            cumsum_columns = columns_sizes + imb_features_size + eng_features  
            full_data = timeseries_cumsum_features(full_data, cumsum_columns)

        #------------------------ get the full data up to current seconds in bucket in date-----------------------------------


            # getting back to test size
            test = full_data.iloc[-test.shape[0]:].copy()
        
        
            #------------------------------------  Global based on stock_id ---------------------------------------
            test.replace([np.inf, -np.inf], np.nan, inplace=True)
            

            if test.currently_scored.iloc[0]:
                if is_lgb:
                    test['target'] = ens_models[0]*make_predictions(lgb_models,test[lgb_features],model='lgb')

                if is_nn:
                    X_test, _           = second_pass_for_nn(test,numerical_features,categorical_features,is_inference=True)
                    test['target'] += ens_models[1]*make_predictions(nn_models, X_test,model='nn').flatten()

                if is_rnn:
                    X_test, _           = second_pass_for_rnn(full_data, rnn_numerical_features, rnn_categorical_features, window_size, is_inference=True)
                    test['target'] += ens_models[2]*make_predictions(rnn_models, X_test,model='nn').flatten()



            if not kaggle:
                total_test = pd.concat([total_test, test], ignore_index=True)

            
            daily_online = pd.concat([daily_online, test], ignore_index=True)


            sample_prediction = pd.merge(sample_prediction.drop(columns='target'), test[['row_id', 'target']], on=['row_id'], how='left')
            sample_prediction['target'].fillna(0, inplace=True)
            sample_prediction['target'].replace([np.inf, -np.inf], 0, inplace=True)

        except Exception as e:
            sample_prediction['target'] = 0

        env.predict(sample_prediction)
        i += 1

loading lgb models...
Done!
loading nn models...


/opt/conda/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 1.3.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Done!
loading rnn models...
Done!
This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.032370 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 58215
[LightGBM] [Info] Number of data points in the train set: 11000, number of used features: 244
[5001]	training's l1: 4.94885
[5002]	training's l1: 4.94748
[5003]	training's l1: 4.94614
[5004]	training's l1: 4.94499
[5005]	training's l1: 4.94365
[5006]	training's l1: 4.94235
[5007]	training's l1: 4.94114
[5008]	training's l1: 4.93975
[5009]	training's l1: 4.93848
[5010]	training's l1: 4.93715
[5011]	training's l1: 4.93581
[5012]	training's l1: 4.93462
[5013]	training's l1: 4.93332
[5014]	training's l1: 4.93198
[5015]	training's l1: 4.93064
[5016]	training's l1: 4.92933
[5017]	training's l1: 4.92806
[5018]	training's l1: 4.92676
[5019]	training's l1: 4.92561
[5020]	training's l1: 4.92431
Epoch 1/4
1/1 [==============================] - 5s 5s/step - loss: 5.1576
Epoch 2/4
1/1 [================

...

Epoch 1/4
3/3 [==============================] - 12s 450ms/step - loss: 5.1534
Epoch 2/4
3/3 [==============================] - 1s 448ms/step - loss: 5.1636
Epoch 3/4
3/3 [==============================] - 1s 446ms/step - loss: 5.1373
Epoch 4/4
3/3 [==============================] - 1s 448ms/step - loss: 5.1200


/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.036268 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 59235
[LightGBM] [Info] Number of data points in the train set: 11000, number of used features: 248
[5021]	training's l1: 5.27298
[5022]	training's l1: 5.27184
[5023]	training's l1: 5.27065
[5024]	training's l1: 5.26958
[5025]	training's l1: 5.26841
[5026]	training's l1: 5.26724
[5027]	training's l1: 5.26607
[5028]	training's l1: 5.26484
[5029]	training's l1: 5.26362
[5030]	training's l1: 5.26248
[5031]	training's l1: 5.26126
[5032]	training's l1: 5.26006
[5033]	training's l1: 5.25898
[5034]	training's l1: 5.25795
[5035]	training's l1: 5.25674
[5036]	training's l1: 5.25571
[5037]	training's l1: 5.2547
[5038]	training's l1: 5.25335
[5039]	training's l1: 5.25231
[5040]	training's l1: 5.25134
Epoch 1/4
1/1 [==============================] - 4s 4s/step - loss: 5.5821
Epoch 2/4
1/1 [=================

Processing groups: 100%|██████████| 200/200 [00:02<00:00, 71.29it/s]


Epoch 1/4
3/3 [==============================] - 12s 448ms/step - loss: 5.5664
Epoch 2/4
3/3 [==============================] - 1s 461ms/step - loss: 5.5794
Epoch 3/4
3/3 [==============================] - 1s 443ms/step - loss: 5.5423
Epoch 4/4
3/3 [==============================] - 1s 442ms/step - loss: 5.5268


In [21]:
#@title check inference

if  is_inference and not kaggle: 

    last_date = True
    if last_date:
        original = test_data.query("date_id >= 480").reset_index()
        loop_test = total_test.query("date_id >= 480").reset_index()
    else:
        original = test_data
        loop_test = total_test


    if is_lgb:
        original['preds'] =  make_predictions(lgb_models, original[features],model='lgb')
        mae = mean_absolute_error(original['target'], original['preds'])
        print(f"Mean Absolute Error lgb: {mae:.4f}")

    if is_nn:
        X_test, y_test  = second_pass_for_nn(original,numerical_features,categorical_features)
        predictions =  make_predictions(nn_models, X_test,model='nn')
        print(f"Mean Absolute Error nn: {mean_absolute_error(y_test, predictions):.4f}")
    
    if is_rnn:
        X_test, y_test  = second_pass_for_rnn(original,rnn_numerical_features,rnn_categorical_features,window_size)
        predictions =  make_predictions(rnn_models, X_test,model='nn')
        print(f"Mean Absolute Error rnn: {mean_absolute_error(y_test, predictions):.4f}")

    mae = mean_absolute_error(original['target'], loop_test['target'])
    print(f"Mean Absolute Error on loop inference : {mae:.4f}")
    

#0.05, 31 full vaidation Mean Absolute Error lgb on origin data 480: 4.7701
# # test discrepencies
# common_columns = loop_test.columns.intersection(original.columns)

# # Step 2: Reorder columns in both DataFrames
# loop_test_common = loop_test[common_columns]
# original_common = original[common_columns]
# loop_test_common.describe().to_csv('loop.csv')
# original_common.describe().to_csv("original.csv")

In [22]:
#@title all preds 


def assign_predictions_and_reorder(test_data, preds):


    test_data['original_order'] = test_data.index
    test_data_sorted = test_data.sort_values(by=['stock_id', 'date_id'])
    test_data_sorted['rnn_preds'] = preds
    test_data_original_order = test_data_sorted.sort_values(by='original_order')
    return test_data_original_order['rnn_preds'].values


def second_pass_for_rnn_sim(df, rnn_numerical_features, rnn_categorical_features, window_size):
    # Check if the DataFrame is empty
    global rnn_scaler,rnn_medians
    
    if df.empty:
        return None, None

    # Work on a copy of the DataFrame to avoid changing the original df
    df_copy = df.copy()

    # Standard scaling for numerical features
    df_copy.fillna(rnn_medians, inplace=True)
    df_copy[rnn_numerical_features] = rnn_scaler.transform(df_copy[rnn_numerical_features])
        
    # Preprocess Data
    df_copy['seconds_in_bucket'] = df_copy['seconds_in_bucket'] / 10
    df_copy['imbalance_buy_sell_flag'] += 1 

    df_copy_batches, df_copy_targets = create_batches(df_copy, window_size, rnn_numerical_features, rnn_categorical_features)

    df_copy_batches = np.array(df_copy_batches)
    df_copy_targets = np.array(df_copy_targets)

    return df_copy_batches, df_copy_targets


if colab:
    simulation_path = "/content/drive/My Drive/optiver/optiver-258-rnn-vals-74/"
else:
    simulation_path = "vals-258/"


simulation_ens = False
if simulation_ens:

    models_path = simulation_path

    lgb_model             = lgb.Booster(model_file=f'{models_path}model_lgb_0.txt')
    lgb_features          = lgb_model.feature_name()
    lgb_preds             = lgb_model.predict(test_data[lgb_features])


    loaded_data          = load_pickle(f'{models_path}all_data.pkl')
    scaler               = loaded_data["scaler"]
    medians              = loaded_data["medians"]
    categorical_features = loaded_data["categorical_features"]
    numerical_features   = loaded_data["numerical_features"]
    nn_model             = load_model(f"{models_path}swish_model_seed_0.h5")
    X_test, _            = second_pass_for_nn(test_data,numerical_features,categorical_features,is_inference = True)
    nn_preds             = nn_model.predict(X_test, batch_size=16384).flatten()



    loaded_data                 = load_pickle(f'{models_path}rnn_all_data.pkl')
    rnn_scaler                  = loaded_data["rnn_scaler"]
    rnn_medians                 = loaded_data["rnn_medians"]
    rnn_categorical_features    = loaded_data["rnn_categorical_features"]
    rnn_numerical_features      = loaded_data["rnn_numerical_features"]
    rnn_model                   = load_model(f"{models_path}rnn_model_seed_0.h5")
    X_test, _                   = second_pass_for_rnn_sim(test_data,rnn_numerical_features,rnn_categorical_features,window_size)
    rnn_preds                   = rnn_model.predict(X_test, batch_size=16384).flatten()
    rnn_preds                   = assign_predictions_and_reorder(test_data,rnn_preds)



    test_data['lgb_preds']  = lgb_preds
    test_data['nn_preds']   = nn_preds
    test_data['rnn_preds']  = rnn_preds


    

In [23]:
if simulation_ens:
    mae_scores = {}

    # List of model predictions
    model_predictions = ['lgb_preds','nn_preds' ,'rnn_preds']

    # Calculate MAE for each model
    for model_pred in model_predictions:
        mae = mean_absolute_error(test_data['target'], test_data[model_pred])
        mae_scores[model_pred] = mae
        print(f"{model_pred} Score on Test: {mae}")
    
    def simple_average_ensemble(test_data):
        test_data['ensemble_pred'] = (test_data['lgb_preds'] + test_data['nn_preds'] + test_data['rnn_preds']) / 3
        return test_data
    
    def weighted_average_ensemble(test_data, weights):

        total_weight = sum(weights.values())
        test_data['ensemble_pred'] = (test_data['lgb_preds'] * weights['lgb'] +  test_data['nn_preds'] * weights['nn'] +  test_data['rnn_preds'] * weights['rnn']) / total_weight
        return test_data


    test_data = simple_average_ensemble(test_data)

    avg_score = mean_absolute_error(test_data['target'], test_data['ensemble_pred'])

    print(f"average Score on Test: {avg_score}")
    
    weights = {'lgb': 0.5, 'nn': 0.25, 'rnn': 0.25}  
    test_data = weighted_average_ensemble(test_data, weights)
    avg_score = mean_absolute_error(test_data['target'], test_data['ensemble_pred'])

    print(f"weighted average Score on Test: {avg_score}")

    hm= (2*(test_data['nn_preds']*test_data['rnn_preds']))/(test_data['nn_preds']+test_data['rnn_preds'])

    avg_score = mean_absolute_error(test_data['target'],.5*test_data['lgb_preds']  + .5*hm )

    print(f"harmonic mean weighted average Score on Test: {avg_score}")


    print("---------------------------")


    test_data_418 =  test_data.query(f"date_id >= 418").reset_index().copy()

    test_data_000 =  test_data.query(f"date_id < 418").reset_index().copy()

    for model_pred in model_predictions:
        mae = mean_absolute_error(test_data_418['target'], test_data_418[model_pred])
        mae_scores[model_pred] = mae
        print(f"{model_pred} Score on Test 418: {mae}")


    test_data_418 = simple_average_ensemble(test_data_418)

    avg_score = mean_absolute_error(test_data_418['target'], test_data_418['ensemble_pred'])

    print(f"average Score on Test: {avg_score}")
    
    weights = {'lgb': 0.5, 'nn': 0.25, 'rnn': 0.25}  
    test_data_418 = weighted_average_ensemble(test_data_418, weights)
    avg_score = mean_absolute_error(test_data_418['target'], test_data_418['ensemble_pred'])

    print(f"weighted average Score on Test 418: {avg_score}")

    print("---------------------------")



In [24]:
# lgb_preds Score on Test: 5.860964545705968
# nn_preds Score on Test: 5.871083373009854
# rnn_preds Score on Test: 5.8716475533769525
# average Score on Test: 5.8557724356023275
# weighted average Score on Test: 5.854561274377742
# ---------------------------
# lgb_preds Score on Test 418: 5.659749933905271
# nn_preds Score on Test 418: 5.669579630390044
# rnn_preds Score on Test 418: 5.670666008919795
# average Score on Test: 5.654617576426593
# weighted average Score on Test: 5.6533975842069975

In [25]:
#@title simulation




import math


def online_pass_for_rnn_sim(df, rnn_numerical_features, rnn_categorical_features, window_size):
    # Check if the DataFrame is empty
    global rnn_scaler,rnn_medians
    
    if df.empty:
        return None, None

    # Work on a copy of the DataFrame to avoid changing the original df
    df_copy = df.copy()

    # Standard scaling for numerical features
    df_copy.fillna(rnn_medians, inplace=True)
    df_copy[rnn_numerical_features] = rnn_scaler.transform(df_copy[rnn_numerical_features])

    # Preprocess Data
    df_copy['seconds_in_bucket'] = df_copy['seconds_in_bucket'] / 10
    df_copy['imbalance_buy_sell_flag'] += 1 


    grouped = df_copy.groupby(['stock_id'])
    all_batches = []
    all_targets = []

    for _, group in tqdm(grouped, desc="Processing groups"):
        # Precompute sequences for the current group
        precomputed_data = precompute_sequences(group, window_size, rnn_numerical_features, rnn_categorical_features)

        # Initialize containers for group sequences and targets
        group_sequences = []
        group_targets = []

        # Iterate over the time steps and retrieve precomputed sequences
        for time_step in range(55):
            sequence, target = get_sequence(precomputed_data, time_step)
            if sequence.size > 0: 
                group_sequences.append(sequence)
                group_targets.append(target)

        # Extend the main batches with the group's sequences and targets
        all_batches.extend(group_sequences)
        all_targets.extend(group_targets)

    df_batches = np.array(all_batches)
    df_targets = np.array(all_targets)


    return df_batches, df_targets

if simulation:
    
    models_path = simulation_path
    online_learning= True

    #-------------------LGB----------------
    initial_learning_rates = [0.001]
    n_estimators_values    = [20]#,30,40]
    num_leaves_values      = [1023]
    max_depth_values       = [2]#,4,6,8,10,12,14]
    #-------------------LGB----------------

    if is_lgb:
        print("loading lgb models...")
        lgb_models_original = []
        for i in range(1):
            loaded_model = lgb.Booster(model_file=f'{models_path}model_lgb_{i}.txt')
            lgb_models_original.append(loaded_model)

        lgb_features = lgb_models_original[0].feature_name()
        print("making prediction on the whole test data....")
        preds = make_predictions(lgb_models_original,test_data[lgb_features],model='lgb')
        mae_original = mean_absolute_error(test_data['target'],preds)
        print(f"lgb Score on Test: {mae_original}")


    #-------------------NN----------------
    nn_init_rate_values = [1e-4]#,2e-4,1e-5,5e-4]
    online_batch_size_values =[16384]
    online_epochs_values = [2]#4,8,10]
    #-------------------NN----------------

    if is_nn:
        print("loading nn models...")
        loaded_data          = load_pickle(f'{models_path}all_data.pkl')
        scaler               = loaded_data["scaler"]
        medians              = loaded_data["medians"]
        categorical_features = loaded_data["categorical_features"]
        numerical_features   = loaded_data["numerical_features"]
        nn_models_original = []
        for i in range(1):
            loaded_model_original = load_model(f"{models_path}swish_model_seed_{i}.h5")
            nn_models_original.append(loaded_model_original)
        print("Done!")

        print("making prediction on the whole test data....")
        X_test, _       = second_pass_for_nn(test_data,numerical_features,categorical_features,is_inference=True)
        preds           = make_predictions(nn_models_original, X_test,model='nn').flatten()
        mae_original    = mean_absolute_error(test_data['target'],preds)
        print(f"nn Score on Test: {mae_original}")


    #-------------------RNN----------------
    rnn_init_rate_values = [2e-4,4e-4]#,2e-4]
    rnn_online_batch_size_values =[2**12,2**10]#,2**11,2**12,2*9]
    rnn_online_epochs_values = [4,6,10]
    #-------------------RNN----------------

    if is_rnn:
        print("loading rnn models...")
        loaded_data          = load_pickle(f'{models_path}rnn_all_data.pkl')
        rnn_scaler               = loaded_data["rnn_scaler"]
        rnn_medians              = loaded_data["rnn_medians"]
        rnn_categorical_features = loaded_data["rnn_categorical_features"]
        rnn_numerical_features   = loaded_data["rnn_numerical_features"]
        rnn_models_original = []
        for i in range(1):
            loaded_model_original = load_model(f"{models_path}rnn_model_seed_{i}.h5")
            rnn_models_original.append(loaded_model_original)
        print("Done!")

        X_test_batches, y_test_batches           = second_pass_for_rnn_sim(test_data,rnn_numerical_features,rnn_categorical_features,window_size)
        preds = make_predictions(rnn_models_original, X_test_batches,model='nn').flatten()
        preds = assign_predictions_and_reorder(test_data,preds)
        mae_original    = mean_absolute_error(test_data['target'],preds)
        print(f"rnn Score on Test: {mae_original}")



    br_count = 200

    results = []


# Nested loop over all parameter combinations
    
    for init_lr in initial_learning_rates:
        for n_estimators in n_estimators_values:
            for num_leaves in num_leaves_values:
                    for max_depth in max_depth_values:

                        for nn_init_rate in nn_init_rate_values:
                            for online_batch_size in online_batch_size_values:
                                for online_epochs in online_epochs_values:
                                    for rnn_init_rate in rnn_init_rate_values:
                                        for rnn_online_batch_size in rnn_online_batch_size_values:
                                            for rnn_online_epochs in rnn_online_epochs_values:

                                                if is_nn:
                                                    print("loading nn models...")
                                                    loaded_data          = load_pickle(f'{models_path}all_data.pkl')
                                                    scaler               = loaded_data["scaler"]
                                                    medians              = loaded_data["medians"]
                                                    categorical_features = loaded_data["categorical_features"]
                                                    numerical_features   = loaded_data["numerical_features"]

                                                    nn_models = []
                                                    for i in range(1):
                                                        loaded_model = load_model(f"{models_path}swish_model_seed_{i}.h5")
                                                        nn_models.append(loaded_model)
                                                    print("Done!")

                                                    nn_models_original = []
                                                    for i in range(1):
                                                        loaded_model_original = load_model(f"{models_path}swish_model_seed_{i}.h5")
                                                        nn_models_original.append(loaded_model_original)
                                                    print("Done!")

                                                if is_rnn:
                                                    print("loading rnn models...")
                                                    loaded_data          = load_pickle(f'{models_path}rnn_all_data.pkl')
                                                    rnn_scaler               = loaded_data["rnn_scaler"]
                                                    rnn_medians              = loaded_data["rnn_medians"]
                                                    rnn_categorical_features = loaded_data["rnn_categorical_features"]
                                                    rnn_numerical_features   = loaded_data["rnn_numerical_features"]

                                                    rnn_models = []
                                                    for i in range(1):
                                                        loaded_model = load_model(f"{models_path}rnn_model_seed_{i}.h5")
                                                        rnn_models.append(loaded_model)
                                                    print("Done!")

                                                    rnn_models_original = []
                                                    for i in range(1):
                                                        loaded_model_original = load_model(f"{models_path}rnn_model_seed_{i}.h5")
                                                        rnn_models_original.append(loaded_model_original)
                                                    print("Done!")

                                                if is_lgb:  
                                                    print("loading lgb models...")
                                                    lgb_models_original = []
                                                    for i in range(1):
                                                        loaded_model = lgb.Booster(model_file=f'{models_path}model_lgb_{i}.txt')
                                                        lgb_models_original.append(loaded_model)


                                                    online_models = []
                                                    for i in range(1):
                                                        loaded_model = lgb.Booster(model_file=f'{models_path}model_lgb_{i}.txt')
                                                        online_models.append(loaded_model)
                                                    
                                                    lgb_features = online_models[0].feature_name()


                                                # Set the parameters for your model
                                                lgb_params['n_estimators'] = n_estimators
                                                lgb_params['num_leaves'] = num_leaves
                                                lgb_params['max_depth'] = max_depth
                                                lgb_params['learning_rate'] = init_lr

                                                mae_values = [] 
                                                loop_test = pd.DataFrame()
                                                subset_cum = pd.DataFrame()
                                                counter = 0

                                                for date_id in tqdm(test_data['date_id'].unique(), desc="Iterating over unique date_ids"):
                                                    

                                                    counter +=1
                                                    #-------------------updating lr decay-------------------
                                                    subset_test = test_data[test_data['date_id'] == date_id]
                                                    
                                                    if is_lgb:
                                                        subset_test['online_preds']   = make_predictions(online_models,subset_test[lgb_features],model='lgb')
                                                        subset_test['original_preds'] = make_predictions(lgb_models_original,subset_test[lgb_features],model='lgb')

                                                    if is_nn:
                                                        X_test, _           = second_pass_for_nn(subset_test,numerical_features,categorical_features,is_inference = True)
                                                        subset_test['online_preds']  = make_predictions(nn_models, X_test,model='nn').flatten()
                                                        subset_test['original_preds'] = make_predictions(nn_models_original, X_test,model='nn').flatten()

                                                    if is_rnn:
                                                        X_test_batches, y_test_batches           = online_pass_for_rnn_sim(subset_test,rnn_numerical_features,rnn_categorical_features,window_size)
                                                        
                                                        online_preds  = make_predictions(rnn_models, X_test_batches,model='nn').flatten()
                                                        original_preds = make_predictions(rnn_models_original, X_test_batches,model='nn').flatten()

                                                        subset_test['online_preds'] = assign_predictions_and_reorder(subset_test,online_preds)
                                                        subset_test['original_preds'] = assign_predictions_and_reorder(subset_test,original_preds)

                                                
                                                    mae_online = mean_absolute_error(subset_test['target'], subset_test['online_preds'])
                                                    mae_original = mean_absolute_error(subset_test['target'], subset_test['original_preds'])

                                                    print(f"-------------------------------------------- Online: Mean Absolute Error lgb: {mae_online:.5f}")
                                                    print(f"-------------------------------------------original: Mean Absolute Error lgb: {mae_original:.5f}")


                                                    mae_values.append({'mae_online': mae_online, 'mae_original': mae_original})

                                                    loop_test = pd.concat([loop_test, subset_test], ignore_index=True)
                                                    subset_cum = pd.concat([subset_cum, subset_test], ignore_index=True)

                                                    if counter >=br_count:
                                                        break

                                                    if online_learning and subset_cum.date_id.nunique() >= 1:

                                                        if is_lgb:
                                                            print(f"==================learning rate = {lgb_params['learning_rate']}==================")
                                                            #----------------------------------------   LGB online learning   ----------------------------------------
                                                            temp_lgbs = []
                                                            for lgb_model in online_models:
                                                                new_data = lgb.Dataset(subset_test[lgb_features], label=subset_test['target'],free_raw_data=False)
                                                                temp_lgb = lgb.train(lgb_params, new_data, init_model=lgb_model,valid_sets=[new_data,new_data],keep_training_booster=True,verbose_eval=5)
                                                                temp_lgbs.append(temp_lgb)

                                                        

                                                            online_models = temp_lgbs.copy()
                                                            #----------------------------------------   LGB online learning   ----------------------------------------
                                                        
                                                        if is_nn:

                                                            print("updating NN model....")
                                                            for nn_model in nn_models:

                                                                # for layer in nn_model.layers[:-5]:
                                                                #     layer.trainable = False
                                                            
                                                                lr_schedule = ExponentialDecay( initial_learning_rate=nn_init_rate, decay_steps=10000, decay_rate=0.9,staircase=True)
                                                                optimizer = Adam(learning_rate=lr_schedule)
                                                                nn_model.compile(optimizer=optimizer, loss = "mean_absolute_error")
                                                                X_online, y_online    = second_pass_for_nn(subset_test,numerical_features,categorical_features,is_inference=True)
                                                                history = nn_model.fit(X_online, y_online, epochs=online_epochs, batch_size=online_batch_size) 

                                                        if is_rnn:

                                                            online_batches, online_targets = online_pass_for_rnn_sim(subset_test, rnn_numerical_features, rnn_categorical_features, window_size)
                    
                                                            for rnn_model in rnn_models:
                                                                lr_schedule = ExponentialDecay( initial_learning_rate=rnn_init_rate, decay_steps=10000, decay_rate=0.8,staircase=True)
                                                                optimizer = Adam(learning_rate=lr_schedule)
                                                                rnn_model.compile(optimizer=optimizer, loss = "mean_absolute_error")
                                                                rnn_model.fit(online_batches, online_targets, epochs=rnn_online_epochs, batch_size=rnn_online_batch_size)




                                                        subset_cum  = pd.DataFrame()


                                                mae_online = mean_absolute_error(loop_test['target'], loop_test['online_preds'])
                                                mae_original = mean_absolute_error(loop_test['target'], loop_test['original_preds'])

                                                print(f"Total Online: Mean Absolute Error lgb: {mae_online:.5f}")
                                                print(f"Total Original: Mean Absolute Error lgb: {mae_original:.5f}")


                                                loop_test_418 =  loop_test.query(f"date_id >= 418").reset_index().copy()
                                                mae_online_418 = mean_absolute_error(loop_test_418['target'], loop_test_418['online_preds'])
                                                mae_original_418 = mean_absolute_error(loop_test_418['target'], loop_test_418['original_preds'])

                                                results.append({
                                                    'initial_learning_rate': init_lr,
                                                    'n_estimators': n_estimators,
                                                    'num_leaves': num_leaves,
                                                    'max_depth':max_depth,

                                                    'nn_init_rate':nn_init_rate,
                                                    'online_batch_size':online_batch_size,
                                                    'online_epochs':online_epochs,

                                                    'rnn_init_rate':rnn_init_rate,
                                                    'rnn_online_batch_size':rnn_online_batch_size,
                                                    'rnn_online_epochs':rnn_online_epochs,

                                                    'mae_online': mae_online,
                                                    'mae_original': mae_original,
                                                    'mae_online_418_end': mae_online_418,
                                                    'mae_original_418_end': mae_original_418
                                                })
                                                pd.DataFrame(results).to_csv(f"{models_path}iter_rnn_result_online.csv", index=False)




    results_df = pd.DataFrame(results)
    results_df.to_csv(f"{models_path}final_result_rnn_online.csv")


    loop_test_418 =  loop_test.query(f"date_id >= 418").reset_index().copy()
    mae_online_418 = mean_absolute_error(loop_test_418['target'], loop_test_418['online_preds'])
    mae_original_418 = mean_absolute_error(loop_test_418['target'], loop_test_418['original_preds'])

    print(f"Total Online: Mean Absolute Error lgb on 418-end : {mae_online_418:.5f}")
    print(f"Total Original: Mean Absolute Error lgb on 418-end: {mae_original_418:.5f}")

# Total Online: Mean Absolute Error lgb on 418-end : 5.65682
# Total Original: Mean Absolute Error lgb on 418-end: 5.65876
